# Transformaciones

In [ ]:
import json
import pandas as pd
import re
from textblob import TextBlob
from ast import literal_eval
from datetime import datetime

### 1- Dataset User_reviews.json
    Contiene datos de las recomendaciones y comentarios de los jugadores

In [ ]:
##Cargamos los datos desde la carpeta Datasets
##Obserbamos que el Dataset tiene columnas con datos anidados en forma de diccionarios por lo que
##deberemos tratarlo linea por linea

# Creamos esta funcion para corregir y cargar cada linea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# lista para almacenar los datos desanidados
data_list = []

# extraemos números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('./Datasets/australian_user_reviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # iteramos sobre cada reseña
            for reseña in entrada['reviews']:
                # estraemos el numero de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1) 

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# crea el DataFrame
df = pd.DataFrame(data_list)



In [ ]:
# Reemplaza los valores NaN por None en el DataFrame
df = df.where(df.notna(), None)
df


In [ ]:
#La columna ['posted'] presenta formato de fecha completo y algunas sin el año, 
# deduzco que el año corresponde al año 2016 en curzo en el momento de la extraccion de los datos
#por lo que realizaremos la extraccion, transformacion y asignacion del año correcto 

# Definimos una función para extraer y convertir la fecha
def extract_date(posted_str):
    if not isinstance(posted_str, str):
        return None
    
    # Elimina el punto y espacios en blanco al principio y al final
    date_str = posted_str.replace('.', '').strip()
    try:
        # Intenta convertir la cadena de fecha en un objeto de fecha utilizando datetime.strptime
        date_obj = datetime.strptime(date_str, '%B %d, %Y').date()
    except ValueError:
        try:
            # Intenta nuevamente con el formato '%B %d' si la conversión falla
            date_obj = datetime.strptime(date_str, '%B %d').date()
            # Si solo tiene mes y día y no se proporcionó el año, asumimos el año 2016
            date_obj = date_obj.replace(year=2016)
        except ValueError:
            # Si no se puede convertir a ninguna de las dos formas, devuelve None
            date_obj = None

    return date_obj

# Aplica la función extract_date a la columna 'posted'
df['posted'] = df['posted'].apply(extract_date)

# Mostrar las primeras filas para verificar
print(df)


In [ ]:
#Eliminamos la columna 'user_url' porque no nos aporta informacion necesaria
df.drop('user_url', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'helpful' porque no nos aporta informacion necesaria
df.drop('helpful', axis=1, inplace=True)
df

In [ ]:
#Eliminamos la columna 'funny' porque no nos aporta informacion necesaria
df.drop('funny', axis=1, inplace=True)
df

### 1a- Analisis de Sentimientos
    realizamos el analisis y adjuntamos la nueva columna al Dataset

In [36]:
#Definimos una funcion que nos permita analizar el sentimiento de las reseñas junto con su 
# recomendación. Esta función evaluará si una reseña es positiva, neutral o negativa, 
# considerando también si la recomendación asociada es verdadera (positiva) o falsa (negativa). 
# Luego, aplicaremos esta función a cada reseña en nuestro DataFrame para obtener 
# un análisis de sentimiento más completo.

def get_sentiment_with_recommend(recommend, review):
    if isinstance(review, str):  # Verifica si el tipo de review es string
        if recommend:  # Verifica si la recomendación es verdadera
            analysis = TextBlob(str(review))  # Convierte a cadena y analiza el texto de la reseña
            polarity = analysis.sentiment.polarity
            if polarity > 0:
                return 2  # Positivo si polaridad > 0
            else:
                return 1  # Neutral si polaridad <= 0 y recommend true
        else:
            return 0  # No recomendado, se asume neutral o negativo
    else:
        return 1  # No hay reseña, se asume neutral

# Convertimos la columna 'review' a cadena (str)
df['review'] = df['review'].astype(str)

# Aplicamos la función a cada review
df['sentiment_analysis'] = df.apply(lambda x: get_sentiment_with_recommend(x['recommend'], x['review']), axis=1)

# Reemplazamos la columna 'review' con la nueva columna 'sentiment_analysis'
df.drop('review', axis=1, inplace=True)
# Renombramos a la columna 'sentiment_analysis' como 'review' 
df.rename(columns={'sentiment_analysis': 'review'}, inplace=True)

print(df)

                 user_id      posted  item_id  recommend  review
0      76561198079601835  2016-05-20      730       True       1
1            MeaTCompany  2016-07-24      730       True       1
2      76561198089393905  2015-02-01    72850       True       1
3      76561198089393905  2014-06-20      440       True       1
4      76561198156664158  2016-06-16   252950       True       2
...                  ...         ...      ...        ...     ...
29633  76561198312638244  2016-07-10       70       True       2
29634  76561198312638244  2016-07-08   362890       True       2
29635        LydiaMorley  2016-07-03   273110       True       2
29636        LydiaMorley  2016-07-20      730       True       2
29637        LydiaMorley  2016-07-02      440       True       2

[29638 rows x 5 columns]


In [38]:
#Guardamos el df resultante en formato parquet 
df.to_parquet('./Datasets/user_reviews.parquet')